## Aggregate Functions

Let us see how to perform aggregations within each group while projecting the raw data that is used to perform the aggregation.

 * We have functions such as `sum`, `avg`, `min`, `max` etc which can be used to aggregate the data.
 * We need to create `WindowSpec` object using `partitionBy` to get aggregations within each group.
 * Typically we don’t need to sort the data to perform aggregations, however if we want to perform cumulative aggregations using rowsBetween, then we have to sort the data using cumulative criteria.
 * Let us try to get total departure delay, minimum departure delay, maximum departure delay and average departure delay for each day for each airport. We will ignore all those flights which are departured early or ontime.

In [ ]:
airlines_path = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
  read. \
  parquet(airlines_path)

In [ ]:
from pyspark.sql.functions import col, lit, lpad, concat

In [ ]:
from pyspark.sql.functions import min, max, sum, avg

In [ ]:
from pyspark.sql.window import Window

In [ ]:
airlines.printSchema()

In [ ]:
spec = Window. \
    partitionBy("FlightDate", "Origin")

In [ ]:
airlines. \
    filter("IsDepDelayed = 'YES' and Cancelled = 0"). \
    select(concat("Year", 
                  lpad("Month", 2, "0"), 
                  lpad("DayOfMonth", 2, "0")
                 ).alias("FlightDate"),
           "Origin",
           "UniqueCarrier",
           "FlightNum",
           "CRSDepTime",
           "IsDepDelayed",
           col("DepDelay").cast("int").alias("DepDelay")
          ). \
    withColumn("DepDelayMin", min("DepDelay").over(spec)). \
    withColumn("DepDelayMax", max("DepDelay").over(spec)). \
    withColumn("DepDelaySum", sum("DepDelay").over(spec)). \
    withColumn("DepDelayAvg", avg("DepDelay").over(spec)). \
    orderBy("FlightDate", "Origin", "DepDelay"). \
    show()